In [1]:
import warnings
warnings.filterwarnings('ignore')

import pickle
from collections.abc import Iterator

import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
from pandas import DataFrame

pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas


from config import *

In [2]:
def get_query(
    user,
    password,
    account,
    warehouse,
    database,
    schema,
    query,
    how="all",
    batch_size=10000,
    conn = None
) -> DataFrame | Iterator[DataFrame]:
    """
    This function will return a pandas dataframe from a query
    """
    if conn == None:
    # Create a connection object
        conn = snowflake.connector.connect(
            user=user,
            password=password,
            account=account,
            warehouse=warehouse,
            database=database,
            schema=schema,
        )
    # Create a cursor object
    cur = conn.cursor()
    # Execute the query
    cur.execute(query)

    if how == "all":
        # Fetch the results
        # Fetch the result set from the cursor and deliver it as the Pandas DataFrame.
        df = cur.fetch_pandas_all()
    elif how == "many":
        # Fetch the results
        df = cur.fetch_pandas_batches(batch_size=batch_size)

    # Close the connection
    # conn.close()
    return df

In [3]:
user = USER
password = PASSWORD
account = ACCOUNT
warehouse = WAREHOUSE
database = 'TEMPORARY_DATA'
schema = 'BIDEM'
uploading_conn = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema
)

In [4]:
df = get_query(
    user=USER,
    password=PASSWORD,
    account=ACCOUNT,
    warehouse=WAREHOUSE,
    database="TEMPORARY_DATA",
    schema="BIDEM",
    query="""
WITH filtered_experience AS (
    SELECT 
        BGI_ONET_NAME, 
        PERSON_ID, 
        ID, 
        BGI_START_DATE, 
        BGI_END_DATE,
        BGI_NO_TITLE_FLAG
    FROM PDL_CLEAN.V4.EXPERIENCE
    WHERE BGI_JOB_COUNTRY = 'United States'
      AND BGI_ONET_CONFIDENCE > 50
)
SELECT *
FROM filtered_experience
WHERE PERSON_ID IN (
    SELECT PERSON_ID
    FROM filtered_experience
    GROUP BY PERSON_ID
    HAVING COUNT(*) > 1 
       AND COUNT(*) < 20
       -- Exclude any person with an experience having BGI_NO_TITLE_FLAG = false
       AND SUM(CASE WHEN BGI_NO_TITLE_FLAG = FALSE THEN 1 ELSE 0 END) = 0
	   -- Exclude any person with an experience having BGI_ONET_NAME = 'Unclassified'
       AND SUM(CASE WHEN BGI_ONET_NAME = 'Unclassified' THEN 1 ELSE 0 END) = 0
       -- Exclude any person with an experience (with both start and end dates) lasting less than a year
       AND SUM(CASE 
                WHEN BGI_END_DATE IS NOT NULL 
                     AND DATEDIFF(day, BGI_START_DATE, BGI_END_DATE) < 365 
                THEN 1 
                ELSE 0 
             END) = 0
)
ORDER BY PERSON_ID, BGI_START_DATE ASC
;
""",
conn=uploading_conn)

In [5]:
df.shape

(54892286, 6)

In [6]:
# Assume df is already loaded. Convert date columns to datetime
df['BGI_START_DATE'] = pd.to_datetime(df['BGI_START_DATE'], errors='coerce')

In [7]:
# before the next step, make BGI_ONET_NAME categorical
df['BGI_ONET_NAME'] = df['BGI_ONET_NAME'].astype('category')

# encode BGI_ONET_NAME
df['BGI_ONET_NAME_ENCODED'] = df['BGI_ONET_NAME'].cat.codes

# save the encoding
onet_name_encoding = df[['BGI_ONET_NAME', 'BGI_ONET_NAME_ENCODED']].drop_duplicates().sort_values('BGI_ONET_NAME_ENCODED').reset_index(drop=True)
onet_name_encoding.to_parquet('data/onet_name_encoding.parquet', index=False)

In [8]:
def create_occupation_sequences(df):
    # Sort the entire dataframe by PERSON_ID and BGI_START_DATE once
    df_sorted = df.sort_values(['PERSON_ID', 'BGI_START_DATE'])
    
    # Group by PERSON_ID and convert each group's BGI_ONET_NAME_ENCODED to a numpy array
    sequences = (
        df_sorted.groupby('PERSON_ID')['BGI_ONET_NAME_ENCODED']
        .apply(lambda x: x.to_numpy(dtype=np.int16))
        .tolist()
    )
    
    return sequences


In [9]:
# Process the dataframe and get sequences
sequences = create_occupation_sequences(df)

In [10]:
# Save sequences as a pickle file
with open('data/occupation_sequences.pkl', 'wb') as f:
    pickle.dump(sequences, f)